In [1]:
import os
import conda
import pandas as pd
import numpy as np
import json
# from shapely.geometry import Polygon as Poly

pd.options.display.max_columns = 250

conda_file_dir = conda.__file__
conda_dir = conda_file_dir.split('lib')[0]
proj_lib = os.path.join(os.path.join(conda_dir, 'share'), 'proj')
os.environ["PROJ_LIB"] = proj_lib

import matplotlib.pyplot as plt
import matplotlib.cm
import matplotlib.colors as colors

import geopandas
from geopandas.tools import sjoin
import geoplot as gplt
import geoplot.crs as gcrs

import folium

In [2]:
# %matplotlib inline

# Databases preparation

## Land Cover

Take Land Cover table from EnviroAtlas, regarding land cover, and reconciliate at HUC8 level since chl-a data for TSI estimation is only available at HUC8 spatial scale. Additionaly, the work of Lee and Bakshi, 2019 (Energy-Water-CO2 Nexus of Fossil Fuel Based Power Generation, book carbon management) support the use of HUC8 scale for Techno-Ecological synergy (TES) analysis.

The fields of the databse are described below:

| Field Name |                                           Data Layer Name                                           |
|:----------:|:---------------------------------------------------------------------------------------------------:|
|   N_INDEX  |                                      Percent natural land cover                                     |
|    PAGT    | Percentage of WBD 12-digit Hydrologic Unit Land area that is classified as agriculture (21, 81, 82) |
|   PFOR90   |                                  Percent forest and woody wetlands                                  |
|   PWETL95  |                                 Percent emergent herbaceous wetlands                                |
|    PFOR    |                                            Percent forest                                           |
|    PWETL   |                                           Percent wetlands                                          |
|    PDEV    |                                        Percent developed area                                       |
|    PAGC    |                                           Percent cropland                                          |
|    PAGP    |                                           Percent pasture                                           |


HUC12 is transformed in HUC8 for data spatial consistency deleting the last 4 digits of the code (Federal Standards and Procedures for the National Watershaed Boundary Dataset (WBD), USGS, 2013). 

To restimate the land cover percentajes the values of each item for the same HUC8 watershed are summed, and the percentajes recalculated again over the new total value for HUC8

In [4]:
LandCover_df = pd.read_csv('LandCover/National_metric_tables_in_CSV/CONUS_metrics_Oct2019_CSV/LandCover.csv', converters={'HUC_12': lambda x: str(x)}) 
LandCover_df

,HUC_12,N_INDEX,PFOR,PWETL,PDEV,PAGT,PAGP,PAGC,PFOR90,PWETL95
0,010100020101,100.000000,81.643204,11.392500,0.00000,0.000000,0.000000,0.00000,92.245598,0.789988
1,010100020102,100.000000,74.108200,12.300500,0.00000,0.000000,0.000000,0.00000,86.143501,0.265201
2,010100020103,100.000000,78.816101,13.675100,0.00000,0.000000,0.000000,0.00000,92.095398,0.395778
3,010100020104,100.000000,72.776901,7.579820,0.00000,0.000000,0.000000,0.00000,80.097702,0.259059
4,010100020105,100.000000,74.281403,13.116500,0.00000,0.000000,0.000000,0.00000,81.182098,6.215810
...,...,...,...,...,...,...,...,...,...,...
82910,181002041305,85.044899,0.000000,0.003772,14.94830,0.006790,0.000000,0.00679,0.000000,0.003772
82911,181002041306,93.744400,0.000000,0.031699,4.96682,1.288760,0.063398,1.22239,0.006934,0.024765
82912,181002041307,81.957397,0.004124,0.116303,2.00107,16.041599,0.004949,15.22330,0.066812,0.053615
82913,181002041308,42.458599,0.002431,0.114263,4.76017,52.781200,2.597670,48.32370,0.082659,0.034036


In [6]:
# File loaded again to avoid overwritting of the LandCover_df variable
LandCover_df = pd.read_csv('LandCover/National_metric_tables_in_CSV/CONUS_metrics_Oct2019_CSV/LandCover.csv', converters={'HUC_12': lambda x: str(x)}) 

# Convert HUC12 to HUC8
LandCover_df['HUC_8'] = LandCover_df['HUC_12'].map(lambda x: str(x)[:-4])
LandCover_df = LandCover_df.drop('HUC_12',1) #0 for rows, 1 for columns

# LandCover_df['HUC_8'] = LandCover_df['HUC_8'].astype(int)
LandCover_df['Total'] = 100

LandCover_df

,N_INDEX,PFOR,PWETL,PDEV,PAGT,PAGP,PAGC,PFOR90,PWETL95,HUC_8,Total
0,100.000000,81.643204,11.392500,0.00000,0.000000,0.000000,0.00000,92.245598,0.789988,01010002,100
1,100.000000,74.108200,12.300500,0.00000,0.000000,0.000000,0.00000,86.143501,0.265201,01010002,100
2,100.000000,78.816101,13.675100,0.00000,0.000000,0.000000,0.00000,92.095398,0.395778,01010002,100
3,100.000000,72.776901,7.579820,0.00000,0.000000,0.000000,0.00000,80.097702,0.259059,01010002,100
4,100.000000,74.281403,13.116500,0.00000,0.000000,0.000000,0.00000,81.182098,6.215810,01010002,100
...,...,...,...,...,...,...,...,...,...,...,...
82910,85.044899,0.000000,0.003772,14.94830,0.006790,0.000000,0.00679,0.000000,0.003772,18100204,100
82911,93.744400,0.000000,0.031699,4.96682,1.288760,0.063398,1.22239,0.006934,0.024765,18100204,100
82912,81.957397,0.004124,0.116303,2.00107,16.041599,0.004949,15.22330,0.066812,0.053615,18100204,100
82913,42.458599,0.002431,0.114263,4.76017,52.781200,2.597670,48.32370,0.082659,0.034036,18100204,100


In [7]:
# Grouped and values sumed, but not percetages yet!
LandCover_dfGrouped = LandCover_df.groupby(['HUC_8']).sum()
LandCover_dfGrouped

,N_INDEX,PFOR,PWETL,PDEV,PAGT,PAGP,PAGC,PFOR90,PWETL95,Total
HUC_8,,,,,,,,,,
01010002,4097.643600,3321.519814,282.046980,2.071450,0.284989,0.088163,0.196827,3583.079910,20.487259,4100
01010003,2291.215496,1733.613396,297.340113,29.957569,178.826970,39.092481,139.734379,2012.386116,18.567774,2500
01010004,5987.979889,4391.402512,917.224541,94.488543,517.531725,77.320543,440.211255,5279.747490,28.879693,6600
01010005,1033.428600,711.407206,244.454398,46.463798,220.107512,30.664473,189.442966,950.056610,5.804847,1300
01010006,2194.817802,1476.533783,317.074897,5.078495,0.103808,0.069891,0.033917,1775.495705,18.113087,2200
...,...,...,...,...,...,...,...,...,...,...
18100100,22623.610382,138.418535,1.148072,252.321961,24.067859,2.714623,21.353237,139.305824,0.260790,22900
18100201,3492.882783,347.273776,5.107398,502.977060,104.140253,13.528867,90.018509,349.129799,3.251223,4100
18100202,1671.564285,17.304101,6.342186,26.862006,1.573615,0.686851,0.886760,21.184718,2.461558,1700


In [8]:
# Percetages calculation
LandCover_dfHUC8 = LandCover_dfGrouped
LandCover_dfHUC8.update(LandCover_dfHUC8.iloc[:, :-1].div(LandCover_dfHUC8.Total, 0))
LandCover_dfHUC8.update(LandCover_dfHUC8.iloc[:, :-1].mul(100))
LandCover_dfHUC8['Total'] = LandCover_dfHUC8['PFOR'] + LandCover_dfHUC8['PWETL'] + LandCover_dfHUC8['PDEV'] + LandCover_dfHUC8['PAGC'] + LandCover_dfHUC8['PAGP']
LandCover_dfHUC8

,N_INDEX,PFOR,PWETL,PDEV,PAGT,PAGP,PAGC,PFOR90,PWETL95,Total
HUC_8,,,,,,,,,,
01010002,99.942527,81.012678,6.879195,0.050523,0.006951,0.002150,0.004801,87.392193,0.499689,87.949347
01010003,91.648620,69.344536,11.893605,1.198303,7.153079,1.563699,5.589375,80.495445,0.742711,89.589518
01010004,90.726968,66.536402,13.897342,1.431645,7.841390,1.171523,6.669868,79.996174,0.437571,89.706779
01010005,79.494508,54.723631,18.804184,3.574138,16.931347,2.358806,14.572536,73.081278,0.446527,94.033295
01010006,99.764446,67.115172,14.412495,0.230841,0.004719,0.003177,0.001542,80.704350,0.823322,81.763227
...,...,...,...,...,...,...,...,...,...,...
18100100,98.793058,0.604448,0.005013,1.101843,0.105100,0.011854,0.093246,0.608322,0.001139,1.816404
18100201,85.192263,8.470092,0.124571,12.267733,2.540006,0.329972,2.195573,8.515361,0.079298,23.387942
18100202,98.327311,1.017888,0.373070,1.580118,0.092566,0.040403,0.052162,1.246160,0.144798,3.063641


In [9]:
# LandCover_dfHUC8['HUC_8']=LandCover_dfHUC8.index
LandCover_dfHUC8.reset_index(level=0, inplace=True)
LandCover_dfHUC8['HUC_6'] = LandCover_dfHUC8['HUC_8'].map(lambda x: str(x)[:-2])
LandCover_dfHUC8

,HUC_8,N_INDEX,PFOR,PWETL,PDEV,PAGT,PAGP,PAGC,PFOR90,PWETL95,Total,HUC_6
0,01010002,99.942527,81.012678,6.879195,0.050523,0.006951,0.002150,0.004801,87.392193,0.499689,87.949347,010100
1,01010003,91.648620,69.344536,11.893605,1.198303,7.153079,1.563699,5.589375,80.495445,0.742711,89.589518,010100
2,01010004,90.726968,66.536402,13.897342,1.431645,7.841390,1.171523,6.669868,79.996174,0.437571,89.706779,010100
3,01010005,79.494508,54.723631,18.804184,3.574138,16.931347,2.358806,14.572536,73.081278,0.446527,94.033295,010100
4,01010006,99.764446,67.115172,14.412495,0.230841,0.004719,0.003177,0.001542,80.704350,0.823322,81.763227,010100
...,...,...,...,...,...,...,...,...,...,...,...,...
2110,18100100,98.793058,0.604448,0.005013,1.101843,0.105100,0.011854,0.093246,0.608322,0.001139,1.816404,181001
2111,18100201,85.192263,8.470092,0.124571,12.267733,2.540006,0.329972,2.195573,8.515361,0.079298,23.387942,181002
2112,18100202,98.327311,1.017888,0.373070,1.580118,0.092566,0.040403,0.052162,1.246160,0.144798,3.063641,181002
2113,18100203,96.836096,2.083096,0.261187,2.199878,0.964020,0.265453,0.698568,2.250339,0.093944,5.508182,181002


In [10]:
# Save the new database
LandCover_dfHUC8.to_csv('DatabasesClean/LandCover_HUC8.csv', index=False)

## 2017 Agricultural Census

The 2017 Agricultural Census is used to determine the distribution of crops on croplands. However, the data provided by the Agricultural Census have a spatial resolution of HUC6 corresponding to each hydrological unit HUC8 to determine the distribution of crops in each portion of crop land corresponding to each hydrological unit HUC8. Area is in acres (2017 census of Agriculture. Watersheds. U.S. Department of Agriculture, 2019).

If two or more crops were harvested from the same land during the year (double cropping), the acres were counted for each crop. Therefore, the total acres of all crops harvested could exceed the acres of cropland harvested. No double cropping is allowed for hay or fruit and nut crops.


In [11]:
# Load csv file
AgriCensus_df = pd.read_csv('AgriCensus2017Watershed/wate_all_tablesCLEANED_TRASPOSED.csv')

AgriCensus_df

,HUC_6,Year,Corn,Soybeans,Small grains,Cotton,Rice,Vegetables,Orchards,Greenhouse,Other
0,H010100,2017,1833.0,1254.0,37038.0,NaN,NaN,51854.0,137.0,11.0,22151.0
1,H010200,2017,9257.0,NaN,1170.0,NaN,NaN,3453.0,412.0,58.0,33209.0
2,H010300,2017,14719.0,NaN,293.0,NaN,NaN,758.0,356.0,195.0,59311.0
3,H010400,2017,4002.0,NaN,NaN,NaN,NaN,1069.0,1059.0,66.0,24189.0
4,H010500,2017,163.0,NaN,NaN,NaN,NaN,1158.0,435.0,154.0,55529.0
...,...,...,...,...,...,...,...,...,...,...,...
366,H200400,2017,NaN,NaN,NaN,NaN,NaN,38.0,45.0,7.0,11.0
367,H200500,2017,NaN,NaN,NaN,NaN,NaN,NaN,647.0,38.0,NaN
368,H200600,2017,3381.0,NaN,NaN,NaN,NaN,5406.0,3840.0,768.0,219.0
369,H200700,2017,876.0,NaN,NaN,NaN,NaN,339.0,3746.0,300.0,183.0


In [12]:
# AgriCensus_df.dropna(axis='index', how ='any', subset=['HUC_6'])
# AgriCensus_df.dropna(axis='index', how ='any', subset=['HUC_6']).to_csv('AgriCensus2017Watershed/wate_all_tablesCLEANED_TRASPOSED.csv',index=False)

In [13]:
# Percetages calculation
AgriCensus_df['Total'] = AgriCensus_df[['Corn', 'Soybeans', 'Small grains', 'Cotton', 'Rice', 'Vegetables', 'Orchards', 'Greenhouse', 'Other']].sum(axis=1)
# AgriCensus_df.iloc[:, 1:].astype(float)

AgriCensus_df

,HUC_6,Year,Corn,Soybeans,Small grains,Cotton,Rice,Vegetables,Orchards,Greenhouse,Other,Total
0,H010100,2017,1833.0,1254.0,37038.0,NaN,NaN,51854.0,137.0,11.0,22151.0,114278.0
1,H010200,2017,9257.0,NaN,1170.0,NaN,NaN,3453.0,412.0,58.0,33209.0,47559.0
2,H010300,2017,14719.0,NaN,293.0,NaN,NaN,758.0,356.0,195.0,59311.0,75632.0
3,H010400,2017,4002.0,NaN,NaN,NaN,NaN,1069.0,1059.0,66.0,24189.0,30385.0
4,H010500,2017,163.0,NaN,NaN,NaN,NaN,1158.0,435.0,154.0,55529.0,57439.0
...,...,...,...,...,...,...,...,...,...,...,...,...
366,H200400,2017,NaN,NaN,NaN,NaN,NaN,38.0,45.0,7.0,11.0,101.0
367,H200500,2017,NaN,NaN,NaN,NaN,NaN,NaN,647.0,38.0,NaN,685.0
368,H200600,2017,3381.0,NaN,NaN,NaN,NaN,5406.0,3840.0,768.0,219.0,13614.0
369,H200700,2017,876.0,NaN,NaN,NaN,NaN,339.0,3746.0,300.0,183.0,5444.0


In [14]:
AgriCensus_df.update(AgriCensus_df.iloc[:, 2:-1].div(AgriCensus_df.Total, 0))
AgriCensus_df.update(AgriCensus_df.iloc[:, 2:-1].mul(100))
AgriCensus_df['Sum'] = AgriCensus_df.iloc[:, 2:-1].sum(axis=1)
AgriCensus_df

,HUC_6,Year,Corn,Soybeans,Small grains,Cotton,Rice,Vegetables,Orchards,Greenhouse,Other,Total,Sum
0,H010100,2017,1.603983,1.097324,32.410438,NaN,NaN,45.375313,0.119883,0.009626,19.383433,114278.0,100.0
1,H010200,2017,19.464244,NaN,2.460102,NaN,NaN,7.260455,0.866292,0.121954,69.826952,47559.0,100.0
2,H010300,2017,19.461339,NaN,0.387402,NaN,NaN,1.002221,0.470700,0.257827,78.420510,75632.0,100.0
3,H010400,2017,13.170973,NaN,NaN,NaN,NaN,3.518183,3.485272,0.217212,79.608359,30385.0,100.0
4,H010500,2017,0.283779,NaN,NaN,NaN,NaN,2.016052,0.757325,0.268111,96.674733,57439.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
366,H200400,2017,NaN,NaN,NaN,NaN,NaN,37.623762,44.554455,6.930693,10.891089,101.0,100.0
367,H200500,2017,NaN,NaN,NaN,NaN,NaN,NaN,94.452555,5.547445,NaN,685.0,100.0
368,H200600,2017,24.834729,NaN,NaN,NaN,NaN,39.709123,28.206258,5.641252,1.608638,13614.0,100.0
369,H200700,2017,16.091109,NaN,NaN,NaN,NaN,6.227039,68.809699,5.510654,3.361499,5444.0,100.0


In [15]:
# Delete H from HUC_6 columns
AgriCensus_df['HUC_6'] = AgriCensus_df['HUC_6'].map(lambda x: str(x)[1:])
AgriCensus_df

,HUC_6,Year,Corn,Soybeans,Small grains,Cotton,Rice,Vegetables,Orchards,Greenhouse,Other,Total,Sum
0,010100,2017,1.603983,1.097324,32.410438,NaN,NaN,45.375313,0.119883,0.009626,19.383433,114278.0,100.0
1,010200,2017,19.464244,NaN,2.460102,NaN,NaN,7.260455,0.866292,0.121954,69.826952,47559.0,100.0
2,010300,2017,19.461339,NaN,0.387402,NaN,NaN,1.002221,0.470700,0.257827,78.420510,75632.0,100.0
3,010400,2017,13.170973,NaN,NaN,NaN,NaN,3.518183,3.485272,0.217212,79.608359,30385.0,100.0
4,010500,2017,0.283779,NaN,NaN,NaN,NaN,2.016052,0.757325,0.268111,96.674733,57439.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
366,200400,2017,NaN,NaN,NaN,NaN,NaN,37.623762,44.554455,6.930693,10.891089,101.0,100.0
367,200500,2017,NaN,NaN,NaN,NaN,NaN,NaN,94.452555,5.547445,NaN,685.0,100.0
368,200600,2017,24.834729,NaN,NaN,NaN,NaN,39.709123,28.206258,5.641252,1.608638,13614.0,100.0
369,200700,2017,16.091109,NaN,NaN,NaN,NaN,6.227039,68.809699,5.510654,3.361499,5444.0,100.0


In [16]:
# Save the new database
AgriCensus_df.to_csv('DatabasesClean/Agricensus.csv', index=False)

## Vegetation nutrient uptake

To determine the nutrients uptake of each type of crop data from USDA (2009) is considered (Waste Management Field Handbook). 

For croplands, the land is divided in the portion occupied by corn, soybeans, small grains, cotton, rice,	vegetables, orchards, greenhouse and other crops (namely oil crops, sugar crops, and fruits). 

For pasture lands the average nutrient uptake and crop yield including the following plants is considered:

    -Alfalfa
    -Switchgrass
    -Wheatgrass

For forests lands the nutrient uptake and crop yield of Northern hardwoods is considered

For developed aread null nutrient uptake is considered

For corn, soybeans, cotton, rice and orchards their specific nutrient uptake values are used. For small grains, vegetables, greenhouse crops, pasture crops and forest average values including the most representatve species are used. (USDA, 2009. Waste Management Field Handbook)b

In [17]:
PlantNutrientUptake_df =  pd.read_csv('PlantNutrientUptake/PlantNutrientUptake.csv', index_col='Crop')
PlantNutrientUptake_df

,ConversionFactor,YieldAcre,PoundsAcre,N,P,K,Ca,Mg,S,Cu,Mn,Zn
Crop,,,,,,,,,,,,
Barley,48.0,50.0,2400.0,1.82,0.34,0.43,0.05,0.1,0.16,0.0016,0.0016,0.0031
BarleyStraw,2000.0,1.0,2000.0,0.75,0.11,1.25,0.40,0.1,0.20,0.0005,0.0160,0.0025
Buckwheat,48.0,30.0,1440.0,1.65,0.31,0.45,0.09,NaN,NaN,0.0009,0.0034,NaN
BuckwheatStraw,2000.0,0.5,1000.0,0.78,0.05,2.26,1.40,NaN,0.01,NaN,NaN,NaN
Corn,56.0,120.0,6720.0,1.61,0.28,0.40,0.02,0.1,0.12,0.0007,0.0011,0.0018
...,...,...,...,...,...,...,...,...,...,...,...,...
Sedges,2000.0,0.8,1600.0,1.79,0.26,NaN,0.66,NaN,NaN,NaN,NaN,NaN
Water hyacinth,NaN,NaN,NaN,NaN,3.65,0.87,3.12,NaN,NaN,NaN,NaN,NaN
Duckweed,NaN,NaN,NaN,3.36,1.00,2.13,NaN,NaN,NaN,NaN,NaN,NaN


For pasture lands the average nutrient uptake and crop yield including the following plants is considered:

    -Alfalfa
    -Switchgrass
    -Wheatgrass

In [18]:
Pasture_df = PlantNutrientUptake_df.loc[['Alfalfa','Switchgrass','Wheatgrass']]
Pasture_df = Pasture_df.mean(axis=0)
Pasture_df.to_csv('DatabasesClean/PasturePlantNutrientUptake.csv', index=True, index_label=['Item'], header=['Value'])
Pasture_df

ConversionFactor    2000.000000
YieldAcre              2.666667
PoundsAcre          5333.333333
N                      1.606667
P                      0.196667
K                      2.150000
Ca                     0.680000
Mg                     0.250000
S                      0.175000
Cu                     0.000800
Mn                     0.005500
Zn                     0.005300
dtype: float64

For forests lands the nutrient uptake and crop yield of Northern hardwoods is considered

In [19]:
Forests_df = PlantNutrientUptake_df.loc[['Northern hardwoods']]
Forests_df = Forests_df.mean(axis=0)
Forests_df.to_csv('DatabasesClean/ForestsPlantNutrientUptake.csv', index=True, index_label=['Item'], header=['Value'])
Forests_df

ConversionFactor      2000.00
YieldAcre               50.00
PoundsAcre          100000.00
N                        0.20
P                        0.02
K                        0.10
Ca                       0.29
Mg                        NaN
S                         NaN
Cu                        NaN
Mn                        NaN
Zn                        NaN
dtype: float64

For small grains the average nutrient uptake and crop yield including the following plants is considered:

    -Wheat
    -WheatStraw
    -Oats
    -OatsStraw
    -Barley
    -BarleyStraw
    -Rye
    -RyeStraw

In [20]:
SmallGrains_df = PlantNutrientUptake_df.loc[['Wheat','WheatStraw','Oats','OatsStraw','Barley','BarleyStraw','Rye','RyeStraw']]
SmallGrains_df = SmallGrains_df.mean(axis=0)
SmallGrains_df.to_csv('DatabasesClean/SmallGrainsPlantNutrientUptake.csv', index=True, index_label=['Item'], header=['Value'])
SmallGrains_df

ConversionFactor    1024.500000
YieldAcre             25.750000
PoundsAcre          2630.000000
N                      1.310000
P                      0.252500
K                      0.812500
Ca                     0.170000
Mg                     0.140000
S                      0.201250
Cu                     0.004613
Mn                     0.006525
Zn                     0.003300
dtype: float64

For vegetables and greehouse the average nutrient uptake and crop yield including the following plants is considered:

    -Bell peppers
    -Beans
    -Cabbage
    -Carrots
    -Cassava
    -Celery
    -Cucumbers
    -Lettuce
    -Onions
    -Peas
    -Potatoes
    -Snap beans
    -Sweet corn
    -Sweet potatoes
    -Table beets
    -Cantaloupe

In [21]:
Vegetables_Greenhouse_df = PlantNutrientUptake_df.loc[['Bell peppers','Beans','Cabbage','Carrots','Cassava','Celery','Cucumbers','Lettuce',
                                                       'Onions','Peas','Potatoes','Snap beans','Sweet corn','Sweet potatoes','Table beets',
                                                       'Cantaloupe']]
Vegetables_Greenhouse_df = Vegetables_Greenhouse_df.mean(axis=0)
Vegetables_Greenhouse_df.to_csv('DatabasesClean/Vegetables_GreenhousePlantNutrientUptake.csv', index=True, index_label=['Item'], header=['Value'])
Vegetables_Greenhouse_df

ConversionFactor     1875.062500
YieldAcre            1104.062500
PoundsAcre          21718.750000
N                       0.744375
P                       0.138125
K                       0.505000
Ca                      0.071000
Mg                      0.084286
S                       0.096000
Cu                      0.000275
Mn                      0.001175
Zn                      0.001040
dtype: float64

For other crops, the average nutrient uptake and crop yield considered includes the most representative crop types not considered in other items, namely oil crops, sugar crops, and fruits:

    -Flax
    -FlaxStraw
    -Oil palm
    -Oil palmStraw
    -Peanuts
    -PeanutsStraw
    -Rapeseed
    -RapeseedStraw
    -Soybeans
    -SoybeansStraw
    -Sunflower
    -SunflowerStraw
    -Apples
    -Bananas
    -Coconuts
    -Grapes
    -Oranges
    -Peaches
    -Pineapple
    -Tomatoes
    -Sugarcane

In [22]:
OtherCrops_df = PlantNutrientUptake_df.loc[['Flax','FlaxStraw','Oil palm','Oil palmStraw','Peanuts','PeanutsStraw','Rapeseed','RapeseedStraw',
                                            'Soybeans','SoybeansStraw','Sunflower','SunflowerStraw','Apples','Bananas','Coconuts','Grapes',
                                            'Oranges','Peaches','Pineapple','Tomatoes','Sugarcane']]
OtherCrops_df = OtherCrops_df.mean(axis=0)
OtherCrops_df.to_csv('DatabasesClean/OtherCropsPlantNutrientUptake.csv', index=True, index_label=['Item'], header=['Value'])
OtherCrops_df

ConversionFactor     1248.571429
YieldAcre            4284.838571
PoundsAcre          18290.952381
N                       1.990476
P                       0.349524
K                       1.180000
Ca                      0.415000
Mg                      0.219048
S                       0.194667
Cu                      0.001067
Mn                      0.006575
Zn                      0.002614
dtype: float64

## Wetlands nutrients regulation

The phosporus uptake due to wetlands is considered as 0.77 gP·m-2·year-1 based in the data reported by Kadlec, 2016 (Kadlec, R.H., 2016
 Large Constructed Wetlands for Phosphorus Control: A Review. Water, 8, 243)

In [23]:
Wetlands_Preg = 0.77*1E-3/0.0002471052 #Kg/Acre

# Watershed retrieval

In [24]:
latitude = 40
longitude = -98

In [25]:
input_point = np.array([float(latitude), float(longitude)]) 

#input_point = np.array([39.014908, -98.010465])

input_point_df = pd.DataFrame(
    {'Name': ['CAFO1'],
     'Latitude': [input_point[0]],
     'Longitude': [input_point[1]]})

#    poly  = geopandas.GeoDataFrame.from_file('cereslibrary/GIS/watershed/huc8sum.shp')
poly  = geopandas.GeoDataFrame.from_file('watershed/huc8sum.shp')
point = geopandas.GeoDataFrame(input_point_df, geometry=geopandas.points_from_xy(input_point_df.Longitude, input_point_df.Latitude))

poly.crs = {'init' :'epsg:4326'}
point.crs = {'init' :'epsg:4326'}

pointInPolys = sjoin(point, poly, how='left')

# HUC8ContPoint = (pointInPolys['HUC_8']).astype(str)

# HUC8_NARS_NLA_FINAL_P  = geopandas.GeoDataFrame.from_file('cereslibrary/GIS/HUC8_NARS_NLA_FINAL_P/HUC8_NARS_NLA_FINAL_P.shp')
# HUC8_NARS_NLA_FINAL_N  = geopandas.GeoDataFrame.from_file('cereslibrary/GIS/HUC8_NARS_NLA_FINAL_N/HUC8_NARS_NLA_FINAL_N.shp')

# HUC8_NARS_NLA_FINAL_P.crs = {'init' :'epsg:4326'}
# HUC8_NARS_NLA_FINAL_N.crs = {'init' :'epsg:4326'}


# =============================================================================
# GIS data retrieval
# =============================================================================

# HUC8_NARS_NLA_FINAL_P_index = HUC8_NARS_NLA_FINAL_P.set_index('HUC_8', drop=False)
# HUC8_NARS_NLA_FINAL_N_index = HUC8_NARS_NLA_FINAL_N.set_index('HUC_8', drop=False)

# TP_GIS = HUC8_NARS_NLA_FINAL_P_index.loc[HUC8ContPoint, 'PTL']
# NH4_GIS = HUC8_NARS_NLA_FINAL_N_index.loc[HUC8ContPoint, 'NH4']

HUC8ContPoint = pointInPolys['HUC_8'].values[0]
HUC8ContPoint

'10250016'

In [26]:
HUC6ContPoint = HUC8ContPoint[:6]
HUC6ContPoint

'102500'

In [27]:
poly['HUC_8'] = poly['HUC_8'].astype(str)
AreaHUC8 = poly.set_index('HUC_8', drop=False)[['AREA_ACRES','HUC_8']]
AreaHUC8.to_csv('DatabasesClean/AreaHUC8.csv')
AreaHUC8


,AREA_ACRES,HUC_8
HUC_8,,
04150306,5.512482e+05,04150306
05010001,1.648037e+06,05010001
05050002,1.078369e+06,05050002
05080001,1.598491e+06,05080001
05090203,9.003527e+05,05090203
...,...,...
17110014,6.449780e+05,17110014
05030104,7.067925e+04,05030104
10250001,1.104509e+06,10250001


In [28]:
poly

,HUC_8,HU_8_NAME,AREA_ACRES,AREA_SQKM,STATES,STORET_COU,STORET_RES,NWIS_COUNT,WSA_CPL_TO,WSA_CPL__1,WSA_CPL__2,WSA_CPL__3,WSA_NAP_TO,WSA_NAP__1,WSA_NAP__2,WSA_NAP__3,WSA_NPL_TO,WSA_NPL__1,WSA_NPL__2,WSA_NPL__3,WSA_SAP_TO,WSA_SAP__1,WSA_SAP__2,WSA_SAP__3,WSA_SPL_TO,WSA_SPL__1,WSA_SPL__2,WSA_SPL__3,WSA_TPL_TO,WSA_TPL__1,WSA_TPL__2,WSA_TPL__3,WSA_UMW_TO,WSA_UMW__1,WSA_UMW__2,WSA_UMW__3,WSA_WMT_TO,WSA_WMT__1,WSA_WMT__2,WSA_WMT__3,WSA_XER_TO,WSA_XER__1,WSA_XER__2,WSA_XER__3,NLA_CGP_TO,NLA_CGP__1,NLA_CGP__2,NLA_CGP__3,NLA_CPE_TO,NLA_CPE__1,NLA_CPE__2,NLA_CPE__3,NLA_GPM_TO,NLA_GPM__1,NLA_GPM__2,NLA_GPM__3,NLA_GPN_TO,NLA_GPN__1,NLA_GPN__2,NLA_GPN__3,NLA_NGL_TO,NLA_NGL__1,NLA_NGL__2,NLA_NGL__3,NLA_SAM_TO,NLA_SAM__1,NLA_SAM__2,NLA_SAM__3,NLA_SEP_TO,NLA_SEP__1,NLA_SEP__2,NLA_SEP__3,NLA_SGL_TO,NLA_SGL__1,NLA_SGL__2,NLA_SGL__3,NLA_TMP_TO,NLA_TMP__1,NLA_TMP__2,NLA_TMP__3,NLA_WMT_TO,NLA_WMT__1,NLA_WMT__2,NLA_WMT__3,NLA_XER_TO,NLA_XER__1,NLA_XER__2,NLA_XER__3,NLCD11A,NLCD11P,NLCD12A,NLCD12P,NLCD21A,NLCD21P,NLCD22A,NLCD22P,NLCD23A,NLCD23P,NLCD24A,NLCD24P,NLCD31A,NLCD31P,NLCD41A,NLCD41P,NLCD42A,NLCD42P,NLCD43A,NLCD43P,NLCD52A,NLCD52P,NLCD71A,NLCD71P,NLCD81A,NLCD81P,NLCD82A,NLCD82P,NLCD90A,NLCD90P,NLCD95A,NLCD95P,LIKELY_TO_,HAS_N_LIMI,HAS_P_LIMI,HAS_N_MONI,HAS_P_MONI,HAS_N_DMR_,HAS_P_DMR_,ATTAINS_30,ATTAINS_TM,Shape_Leng,Shape_Area,geometry
0,04150306,St. Regis,5.512482e+05,2230.822092,NY,5.0,156.0,4.0,0.0,0.0,0.0,0.0,329.0,441.0,8.2,15.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,674.0,1174.0,16.5,36.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,828.0,1410.0,24.0,102.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67.4973,0.030497,0.0000,0.000000,25.6329,0.011582,2.6937,0.001217,0.3033,0.000137,0.0558,0.000025,1.3869,0.000627,1121.7753,0.506844,362.9259,0.163978,50.8455,0.022973,64.2096,0.029011,20.5236,0.009273,83.8224,0.037873,33.8562,0.015297,352.4211,0.159232,25.3251,0.011442,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,10.0,2.739500,0.253010,"POLYGON ((-74.43570 44.34983, -74.43585 44.349..."
1,05010001,Upper Allegheny,1.648037e+06,6669.368806,"NY,PA",20.0,1681.0,25.0,0.0,0.0,0.0,0.0,329.0,441.0,8.2,15.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,674.0,1174.0,16.5,36.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,828.0,1410.0,24.0,102.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59.3739,0.008902,0.0000,0.000000,170.2944,0.025534,38.0439,0.005704,14.2290,0.002133,2.5335,0.000380,16.0839,0.002412,4270.6368,0.640331,302.5584,0.045365,485.9163,0.072857,331.2000,0.049659,51.3369,0.007697,566.8920,0.084999,217.1736,0.032563,129.0528,0.019350,14.1255,0.002118,17.0,12.0,2.0,7.0,13.0,15.0,15.0,12.0,6.0,6.173681,0.724405,"POLYGON ((-79.06824 41.77853, -79.07655 41.778..."
2,05050002,Middle New,1.078369e+06,4364.005019,"VA,WV",70.0,355.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,296.0,535.0,17.8,24.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.1091,0.005753,0.0000,0.000000,193.1715,0.044261,79.0614,0.018115,34.8129,0.007977,5.5368,0.001269,5.3442,0.001225,2902.8690,0.665131,213.4917,0.048917,82.2726,0.018851,3.9078,0.000895,116.5257,0.026699,685.1295,0.156983,14.9175,0.003418,1.6668,0.000382,0.5238,0.000120,32.0,11.0,0.0,21.0,22.0,23.0,22.0,0.0,4.0,4.452832,0.443767,"POLYGON ((-81.52041 37.19589, -81.50728 37.202..."
3,05080001,Upper Great Miami,1.598491e+06,6468.862509,"IN,OH",1.0,3.0,146.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1750.0,3210.0,165.0,338.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

# HUC8 Databases differences

In [29]:
# HUC8 Databases differences
Pruebas_AreaHUC8 = pd.DataFrame(AreaHUC8['HUC_8'])
Pruebas_AreaHUC8 = Pruebas_AreaHUC8.reset_index(drop=True)
Pruebas_LandCover_dfHUC8 = pd.DataFrame(pd.read_csv('DatabasesClean/LandCover_HUC8.csv', converters={'HUC_8': lambda x: str(x), 'HUC_6': lambda x: str(x)})['HUC_8'])
merged = Pruebas_AreaHUC8.merge(Pruebas_LandCover_dfHUC8, indicator=True, how='outer')
merged['HUC_6'] = merged['HUC_8'].map(lambda x: str(x)[:-2])
merged[merged['_merge'] == 'left_only'].to_csv('DatabasesClean/diference.csv')
merged[merged['_merge'] == 'left_only']

,HUC_8,_merge,HUC_6
155,02010001,left_only,020100
316,18030008,left_only,180300
341,18020117,left_only,180201
427,18020107,left_only,180201
482,18060011,left_only,180600
533,18020119,left_only,180201
550,09030004,left_only,090300
646,09010001,left_only,090100
674,09020313,left_only,090203
682,10160010,left_only,101600


# Estimation of the nutrients uptaked by the watershed

Due to problemas with data reconcilitation between EnviroAtlas Land Cover and Agricensus databases, and HUC8 shapefile from EPA NPDAT database, there are no available data for the following HUC8 watersheds:

02010001, 18030008, 18020117, 18020107, 18060011, 18020119, 09030004, 09010001, 09020313, 010160010, 18060012, 02010003, 18100200, 18020102, 18020106, 18020103, 18020101, 18010111, 02010006, 02010007, 02010008, 02060008, 02060009, 02060007, 02060010, 03030001, 18020105, 18020114, 18020112, 18020109, 02010002, 01110000, 10010002, 01010001, 18020113, 18020108, 18020110, 02010005, 18020127, 18020124, 18040005, 18020120, 18030011, 14010006, 18060001, 03020106, 18020118, 18040004, 10010001, 02010004

APLICABLE TO AGRICENSUS AND LAND COVER DATASETS

In [30]:
LandCover_dfHUC8 = pd.read_csv('DatabasesClean/LandCover_HUC8_reconciliated.csv', converters={'HUC_8': lambda x: str(x), 'HUC_6': lambda x: str(x)})
LandCover_dfHUC8 = LandCover_dfHUC8.set_index('HUC_8')
LandCover_dfHUC8

,N_INDEX,PFOR,PWETL,PDEV,PAGT,PAGP,PAGC,PFOR90,PWETL95,Total,HUC_6
HUC_8,,,,,,,,,,,
01010002,99.942527,81.012678,6.879195,0.050523,0.006951,0.002150,0.004801,87.392193,0.499689,87.949347,010100
01010003,91.648620,69.344536,11.893605,1.198303,7.153079,1.563699,5.589375,80.495445,0.742711,89.589518,010100
01010004,90.726968,66.536402,13.897342,1.431645,7.841390,1.171523,6.669868,79.996174,0.437571,89.706779,010100
01010005,79.494508,54.723631,18.804184,3.574138,16.931347,2.358806,14.572536,73.081278,0.446527,94.033295,010100
01010006,99.764446,67.115172,14.412495,0.230841,0.004719,0.003177,0.001542,80.704350,0.823322,81.763227,010100
...,...,...,...,...,...,...,...,...,...,...,...
03020106,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,030201
18020118,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,180201
18040004,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,180400


In [31]:
LandCover_Point = LandCover_dfHUC8.loc[HUC8ContPoint]
LandCover_Point

N_INDEX        42.16
PFOR         1.89975
PWETL        1.25649
PDEV         4.08933
PAGT         53.7506
PAGP         2.18905
PAGC         51.5616
PFOR90       3.08338
PWETL95    0.0728693
Total        60.9962
HUC_6         102500
Name: 10250016, dtype: object

In [32]:
AgriCensus_dfHUC6 = pd.read_csv('DatabasesClean/Agricensus_reconciliated.csv', converters={'HUC_6': lambda x: str(x)})
AgriCensus_dfHUC6 = AgriCensus_dfHUC6.set_index('HUC_6')
AgriCensus_dfHUC6 = AgriCensus_dfHUC6.fillna(0)
AgriCensus_dfHUC6

,Year,Corn,Soybeans,Small grains,Cotton,Rice,Vegetables,Orchards,Greenhouse,Other,Total,Sum
HUC_6,,,,,,,,,,,,
010100,2017.0,1.603983,1.097324,32.410438,0.0,0.0,45.375313,0.119883,0.009626,19.383433,114278.0,100.0
010200,2017.0,19.464244,0.000000,2.460102,0.0,0.0,7.260455,0.866292,0.121954,69.826952,47559.0,100.0
010300,2017.0,19.461339,0.000000,0.387402,0.0,0.0,1.002221,0.470700,0.257827,78.420510,75632.0,100.0
010400,2017.0,13.170973,0.000000,0.000000,0.0,0.0,3.518183,3.485272,0.217212,79.608359,30385.0,100.0
010500,2017.0,0.283779,0.000000,0.000000,0.0,0.0,2.016052,0.757325,0.268111,96.674733,57439.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...
011100,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
020100,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
100100,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0


In [33]:
AgriCensus_Point = AgriCensus_dfHUC6.loc[HUC6ContPoint]
AgriCensus_Point

Year            2.017000e+03
Corn            4.628742e+01
Soybeans        1.303660e+01
Small grains    2.681215e+01
Cotton          0.000000e+00
Rice            0.000000e+00
Vegetables      1.369653e-01
Orchards        1.337244e-03
Greenhouse      3.272315e-03
Other           1.372225e+01
Total           6.356356e+06
Sum             1.000000e+02
Name: 102500, dtype: float64

In [34]:
# Import databases
PlantNutrientUptake_df = pd.read_csv('DatabasesClean/PlantNutrientUptake.csv', index_col='Crop')
ForestsPlantNutrientUptake = pd.read_csv('DatabasesClean/ForestsPlantNutrientUptake.csv', index_col='Item')
SmallGrainsPlantNutrientUptake = pd.read_csv('DatabasesClean/SmallGrainsPlantNutrientUptake.csv', index_col='Item')
VegetablesPlantNutrientUptake = pd.read_csv('DatabasesClean/Vegetables_GreenhousePlantNutrientUptake.csv', index_col='Item')
OtherCropsPlantNutrientUptake = pd.read_csv('DatabasesClean/OtherCropsPlantNutrientUptake.csv', index_col='Item')
PasturePlantNutrientUptake = pd.read_csv('DatabasesClean/PasturePlantNutrientUptake.csv', index_col='Item')

Nutrients uptake for each land type is determined as follows:

$P_{uptake} = Area_{total} \cdot \sum_{j} \left(LandCoverFrac_{j} \cdot \sum_{i} \left( FracCrop_{i} \cdot \left(Yield_{i} \cdot P_{content \ i} \right) \right) \right), \ j 	\in \{\text{Land uses}\}, \ i \in \{\text{Crops}\} $

For croplands, the land is divided in the portion occupied by corn, soybeans, small grains, cotton, rice, vegetables, orchards, greenhouse and other crops (namely oil crops, sugar crops, and fruits). 

In [35]:
# Kg/Acre
Puptake_PFOR = 0.4535924*LandCover_Point['PFOR']/100 * ForestsPlantNutrientUptake.loc['P'].astype(float)/100 * ForestsPlantNutrientUptake.loc['PoundsAcre'].astype(float)
Puptake_PWETL = LandCover_Point['PWETL']/100 * Wetlands_Preg
Puptake_PAGC= 0.4535924*LandCover_Point['PAGC']/100 * (AgriCensus_Point['Corn']/100 * 
                                         (PlantNutrientUptake_df.loc['Corn','P'].astype(float)/100 * PlantNutrientUptake_df.loc['Corn','PoundsAcre'].astype(float)+
                                         PlantNutrientUptake_df.loc['CornStraw','P'].astype(float)/100 * PlantNutrientUptake_df.loc['CornStraw','PoundsAcre'].astype(float))
                                        +
                                         AgriCensus_Point['Soybeans']/100 * 
                                         (PlantNutrientUptake_df.loc['Soybeans','P'].astype(float)/100 * PlantNutrientUptake_df.loc['Soybeans','PoundsAcre'].astype(float)+
                                         PlantNutrientUptake_df.loc['SoybeansStraw','P'].astype(float)/100 * PlantNutrientUptake_df.loc['SoybeansStraw','PoundsAcre'].astype(float))
                                          +
                                         AgriCensus_Point['Small grains']/100 * 
                                         (SmallGrainsPlantNutrientUptake.loc['P'].astype(float)/100 * SmallGrainsPlantNutrientUptake.loc['PoundsAcre'].astype(float))
                                         +
                                         AgriCensus_Point['Cotton']/100 * 
                                         (PlantNutrientUptake_df.loc['Cotton','P'].astype(float)/100 * PlantNutrientUptake_df.loc['Cotton','PoundsAcre'].astype(float)+
                                         PlantNutrientUptake_df.loc['CottonStraw','P'].astype(float)/100 * PlantNutrientUptake_df.loc['CottonStraw','PoundsAcre'].astype(float))
                                         +
                                         AgriCensus_Point['Rice']/100 * 
                                         (PlantNutrientUptake_df.loc['Rice','P'].astype(float)/100 * PlantNutrientUptake_df.loc['Rice','PoundsAcre'].astype(float)+
                                         PlantNutrientUptake_df.loc['RiceStraw','P'].astype(float)/100 * PlantNutrientUptake_df.loc['RiceStraw','PoundsAcre'].astype(float))
                                         +
                                         AgriCensus_Point['Vegetables']/100 * 
                                         (VegetablesPlantNutrientUptake.loc['P'].astype(float) * VegetablesPlantNutrientUptake.loc['PoundsAcre'].astype(float))
                                         +
                                         AgriCensus_Point['Orchards']/100 * 
                                         (PlantNutrientUptake_df.loc['Apples','P'].astype(float)/100 * PlantNutrientUptake_df.loc['Apples','PoundsAcre'].astype(float))
                                         +
                                         AgriCensus_Point['Greenhouse']/100 * 
                                         (VegetablesPlantNutrientUptake.loc['P'].astype(float)/100 * VegetablesPlantNutrientUptake.loc['PoundsAcre'].astype(float))
                                         +
                                         AgriCensus_Point['Other']/100 * 
                                         (OtherCropsPlantNutrientUptake.loc['P'].astype(float)/100 * OtherCropsPlantNutrientUptake.loc['PoundsAcre'].astype(float))
                                        )

Puptake_PAGP = 0.4535924*LandCover_Point['PAGP']/100 * PasturePlantNutrientUptake.loc['P'].astype(float)/100 * PasturePlantNutrientUptake.loc['PoundsAcre'].astype(float)
Puptake_PDEV = 0.4535924*LandCover_Point['PDEV']

In [36]:
TotalHUC_PUptake = AreaHUC8.loc[HUC8ContPoint, 'AREA_ACRES'].astype(float)*float(Puptake_PFOR+Puptake_PWETL+Puptake_PAGC+Puptake_PAGP+Puptake_PDEV)
TotalHUC_PUptake
# AreaHUC8.loc[HUC8ContPoint, 'AREA_ACRES']
# AreaHUC8.loc[HUC8ContPoint]

12482911.021892847

In [37]:
#Check with MRW watershed 05040004
LandCover_dfHUC8 = pd.read_csv('DatabasesClean/LandCover_HUC8_reconciliated.csv', converters={'HUC_8': lambda x: str(x), 'HUC_6': lambda x: str(x)})
LandCover_dfHUC8 = LandCover_dfHUC8.set_index('HUC_8')
LandCover_Point = LandCover_dfHUC8.loc['11120202']

AgriCensus_dfHUC6 = pd.read_csv('DatabasesClean/Agricensus.csv', converters={'HUC_6': lambda x: str(x)})
AgriCensus_dfHUC6 = AgriCensus_dfHUC6.set_index('HUC_6')
AgriCensus_dfHUC6 = AgriCensus_dfHUC6.fillna(0)
AgriCensus_Point = AgriCensus_dfHUC6.loc['111202']

poly  = geopandas.GeoDataFrame.from_file('watershed/huc8sum.shp')
AreaHUC8 = poly.set_index('HUC_8', drop=False)[['AREA_ACRES','HUC_8']]

PlantNutrientUptake_df = pd.read_csv('DatabasesClean/PlantNutrientUptake.csv', index_col='Crop')
ForestsPlantNutrientUptake = pd.read_csv('DatabasesClean/ForestsPlantNutrientUptake.csv', index_col='Item')
SmallGrainsPlantNutrientUptake = pd.read_csv('DatabasesClean/SmallGrainsPlantNutrientUptake.csv', index_col='Item')
VegetablesPlantNutrientUptake = pd.read_csv('DatabasesClean/Vegetables_GreenhousePlantNutrientUptake.csv', index_col='Item')
OtherCropsPlantNutrientUptake = pd.read_csv('DatabasesClean/OtherCropsPlantNutrientUptake.csv', index_col='Item')
PasturePlantNutrientUptake = pd.read_csv('DatabasesClean/PasturePlantNutrientUptake.csv', index_col='Item')

Wetlands_Preg = 0.77*1E-3/0.0002471052 #Kg/Acre

# P uptake forest
Puptake_PFOR = 0.4535924*LandCover_Point['PFOR']/100 * ForestsPlantNutrientUptake.loc['P'].astype(float)/100 * ForestsPlantNutrientUptake.loc['PoundsAcre'].astype(float)
# P uptake wetlands
Puptake_PWETL = LandCover_Point['PWETL']/100 * Wetlands_Preg
# P uptake croplands
Puptake_PAGC= 0.4535924*LandCover_Point['PAGC']/100 * (AgriCensus_Point['Corn']/100 * 
                                         (PlantNutrientUptake_df.loc['Corn','P'].astype(float)/100 * PlantNutrientUptake_df.loc['Corn','PoundsAcre'].astype(float)+
                                         PlantNutrientUptake_df.loc['CornStraw','P'].astype(float)/100 * PlantNutrientUptake_df.loc['CornStraw','PoundsAcre'].astype(float))
                                        +
                                         AgriCensus_Point['Soybeans']/100 * 
                                         (PlantNutrientUptake_df.loc['Soybeans','P'].astype(float)/100 * PlantNutrientUptake_df.loc['Soybeans','PoundsAcre'].astype(float)+
                                         PlantNutrientUptake_df.loc['SoybeansStraw','P'].astype(float)/100 * PlantNutrientUptake_df.loc['SoybeansStraw','PoundsAcre'].astype(float))
                                          +
                                         AgriCensus_Point['Small grains']/100 * 
                                         (SmallGrainsPlantNutrientUptake.loc['P'].astype(float)/100 * SmallGrainsPlantNutrientUptake.loc['PoundsAcre'].astype(float))
                                         +
                                         AgriCensus_Point['Cotton']/100 * 
                                         (PlantNutrientUptake_df.loc['Cotton','P'].astype(float)/100 * PlantNutrientUptake_df.loc['Cotton','PoundsAcre'].astype(float)+
                                         PlantNutrientUptake_df.loc['CottonStraw','P'].astype(float)/100 * PlantNutrientUptake_df.loc['CottonStraw','PoundsAcre'].astype(float))
                                         +
                                         AgriCensus_Point['Rice']/100 * 
                                         (PlantNutrientUptake_df.loc['Rice','P'].astype(float)/100 * PlantNutrientUptake_df.loc['Rice','PoundsAcre'].astype(float)+
                                         PlantNutrientUptake_df.loc['RiceStraw','P'].astype(float)/100 * PlantNutrientUptake_df.loc['RiceStraw','PoundsAcre'].astype(float))
                                         +
                                         AgriCensus_Point['Vegetables']/100 * 
                                         (VegetablesPlantNutrientUptake.loc['P'].astype(float) * VegetablesPlantNutrientUptake.loc['PoundsAcre'].astype(float))
                                         +
                                         AgriCensus_Point['Orchards']/100 * 
                                         (PlantNutrientUptake_df.loc['Apples','P'].astype(float)/100 * PlantNutrientUptake_df.loc['Apples','PoundsAcre'].astype(float))
                                         +
                                         AgriCensus_Point['Greenhouse']/100 * 
                                         (VegetablesPlantNutrientUptake.loc['P'].astype(float)/100 * VegetablesPlantNutrientUptake.loc['PoundsAcre'].astype(float))
                                         +
                                         AgriCensus_Point['Other']/100 * 
                                         (OtherCropsPlantNutrientUptake.loc['P'].astype(float)/100 * OtherCropsPlantNutrientUptake.loc['PoundsAcre'].astype(float))
                                        )
# P uptake pasture
Puptake_PAGP = 0.4535924*LandCover_Point['PAGP']/100 * PasturePlantNutrientUptake.loc['P'].astype(float)/100 * PasturePlantNutrientUptake.loc['PoundsAcre'].astype(float)
# P uptake development areas
Puptake_PDEV = 0.4535924*LandCover_Point['PDEV']/100*0

TotalHUC_PUptake = AreaHUC8.loc['11120202', 'AREA_ACRES'].astype(float)*float(Puptake_PFOR+Puptake_PWETL+Puptake_PAGC+Puptake_PAGP+Puptake_PDEV)
# TotalHUC_PUptake = AreaHUC8.loc['05040004', 'AREA_ACRES'].astype(float)*float(Puptake_PFOR+Puptake_PWETL+Puptake_PAGC+Puptake_PAGP+Puptake_PDEV)
TotalHUC_PUptake/364

5141.141278766131

In [38]:
type(TotalHUC_PUptake)


numpy.float64

# Module definition

In [39]:
# Joining everything in a function
# TotalHUC_PUptake_totalHUCs = []

def PUptake(HUC8ContPoint, HUC6ContPoint):
#     input_point = np.array([float(latitude), float(longitude)]) 

#     #input_point = np.array([39.014908, -98.010465])

#     input_point_df = pd.DataFrame(
#         {'Name': ['CAFO1'],
#          'Latitude': [input_point[0]],
#          'Longitude': [input_point[1]]})

#     poly  = geopandas.GeoDataFrame.from_file('watershed/huc8sum.shp')
#     point = geopandas.GeoDataFrame(input_point_df, geometry=geopandas.points_from_xy(input_point_df.Longitude, input_point_df.Latitude))

#     poly.crs = {'init' :'epsg:4326'}
#     point.crs = {'init' :'epsg:4326'}

#     pointInPolys = sjoin(point, poly, how='left')

#     HUC8ContPoint = pointInPolys['HUC_8'].values[0]
    
#   LandCover_dfHUC8 = LandCover_dfHUC8.set_index('HUC_8')
    LandCover_Point = LandCover_dfHUC8.loc[HUC8ContPoint]
    AgriCensus_Point = AgriCensus_dfHUC6.loc[HUC6ContPoint]
    
    
    # P uptake forest
    Puptake_PFOR = AreaHUC8.loc[HUC8ContPoint, 'AREA_ACRES'].astype(float) * 0.4535924*LandCover_Point['PFOR'].astype(float)/100 * ForestsPlantNutrientUptake.loc['P'].astype(float)/100 * ForestsPlantNutrientUptake.loc['PoundsAcre'].astype(float)
    # P uptake wetlands
    Puptake_PWETL = AreaHUC8.loc[HUC8ContPoint, 'AREA_ACRES'].astype(float) * LandCover_Point['PWETL'].astype(float)/100 * Wetlands_Preg
    # P uptake croplands
    Puptake_PAGC= AreaHUC8.loc[HUC8ContPoint, 'AREA_ACRES'].astype(float) * 0.4535924*LandCover_Point['PAGC'].astype(float)/100 * (AgriCensus_Point['Corn'].astype(float)/100 * 
                                             (PlantNutrientUptake_df.loc['Corn','P'].astype(float)/100 * PlantNutrientUptake_df.loc['Corn','PoundsAcre'].astype(float)+
                                             PlantNutrientUptake_df.loc['CornStraw','P'].astype(float)/100 * PlantNutrientUptake_df.loc['CornStraw','PoundsAcre'].astype(float))
                                             +
                                             AgriCensus_Point['Soybeans'].astype(float)/100 * 
                                             (PlantNutrientUptake_df.loc['Soybeans','P'].astype(float)/100 * PlantNutrientUptake_df.loc['Soybeans','PoundsAcre'].astype(float)+
                                             PlantNutrientUptake_df.loc['SoybeansStraw','P'].astype(float)/100 * PlantNutrientUptake_df.loc['SoybeansStraw','PoundsAcre'].astype(float))
                                             +
                                             AgriCensus_Point['Small grains'].astype(float)/100 * 
                                             (SmallGrainsPlantNutrientUptake.loc['P'].astype(float)/100 * SmallGrainsPlantNutrientUptake.loc['PoundsAcre'].astype(float))
                                             +
                                             AgriCensus_Point['Cotton'].astype(float)/100 * 
                                             (PlantNutrientUptake_df.loc['Cotton','P'].astype(float)/100 * PlantNutrientUptake_df.loc['Cotton','PoundsAcre'].astype(float)+
                                             PlantNutrientUptake_df.loc['CottonStraw','P'].astype(float)/100 * PlantNutrientUptake_df.loc['CottonStraw','PoundsAcre'].astype(float))
                                             +
                                             AgriCensus_Point['Rice'].astype(float)/100 * 
                                             (PlantNutrientUptake_df.loc['Rice','P'].astype(float)/100 * PlantNutrientUptake_df.loc['Rice','PoundsAcre'].astype(float)+
                                             PlantNutrientUptake_df.loc['RiceStraw','P'].astype(float)/100 * PlantNutrientUptake_df.loc['RiceStraw','PoundsAcre'].astype(float))
                                             +
                                             AgriCensus_Point['Vegetables'].astype(float)/100 * 
                                             (VegetablesPlantNutrientUptake.loc['P'].astype(float) * VegetablesPlantNutrientUptake.loc['PoundsAcre'].astype(float))
                                             +
                                             AgriCensus_Point['Orchards'].astype(float)/100 * 
                                             (PlantNutrientUptake_df.loc['Apples','P'].astype(float)/100 * PlantNutrientUptake_df.loc['Apples','PoundsAcre'].astype(float))
                                             +
                                             AgriCensus_Point['Greenhouse'].astype(float)/100 * 
                                             (VegetablesPlantNutrientUptake.loc['P'].astype(float)/100 * VegetablesPlantNutrientUptake.loc['PoundsAcre'].astype(float))
                                             +
                                             AgriCensus_Point['Other'].astype(float)/100 * 
                                             (OtherCropsPlantNutrientUptake.loc['P'].astype(float)/100 * OtherCropsPlantNutrientUptake.loc['PoundsAcre'].astype(float))
                                             )

    # P uptake pasture
    Puptake_PAGP = AreaHUC8.loc[HUC8ContPoint, 'AREA_ACRES'].astype(float) * 0.4535924*LandCover_Point['PAGP'].astype(float)/100 * PasturePlantNutrientUptake.loc['P'].astype(float)/100 * PasturePlantNutrientUptake.loc['PoundsAcre'].astype(float)
    # P uptake development areas
    Puptake_PDEV = AreaHUC8.loc[HUC8ContPoint, 'AREA_ACRES'].astype(float) * 0.4535924*LandCover_Point['PDEV'].astype(float)/100 * 0
    
    # Total P uptake kg/acre
    TotalHUC_PUptake = (Puptake_PFOR+Puptake_PWETL+Puptake_PAGC+Puptake_PAGP+Puptake_PDEV)
    
    # Anthropogenic P uptake
    AnthropogenicHUC_PUptake = (Puptake_PWETL+Puptake_PAGC+Puptake_PAGP+Puptake_PDEV)
    
#    TotalHUC_PUptake = TotalHUC_PUptake.to_numpy()
    TotalHUC_PUptake = TotalHUC_PUptake.item()
    TotalHUC_PUptake_totalHUCs.append(TotalHUC_PUptake)
    
    AnthropogenicHUC_PUptake = AnthropogenicHUC_PUptake.item()
    AnthropogenicHUC_PUptake_totalHUCs.append(AnthropogenicHUC_PUptake)
    
    Puptake_PFOR_HUC = Puptake_PFOR.item()
    Puptake_PFOR_HUC_totalHUCs.append(Puptake_PFOR_HUC)
    
    Puptake_PWETL_HUC = Puptake_PWETL.item()
    Puptake_PWETL_HUC_totalHUCs.append(Puptake_PWETL_HUC)
    
    Puptake_PAGC_HUC = Puptake_PAGC.item()
    Puptake_PAGC_HUC_totalHUCs.append(Puptake_PAGC_HUC)
    
    Puptake_PAGP_HUC =  Puptake_PAGP.item()
    Puptake_PAGP_HUC_totalHUCs.append(Puptake_PAGP_HUC)
    
    Puptake_PDEV_HUC = Puptake_PDEV.item()
    Puptake_PDEV_HUC_totalHUCs.append(Puptake_PDEV_HUC)
                                      
    Area_HUC = AreaHUC8.loc[HUC8ContPoint, 'AREA_ACRES'].item()
    Area_HUC_totalHUCs.append(Area_HUC)
#     return TotalHUC_PUptake

In [40]:
LandCover_dfHUC8 = pd.read_csv('DatabasesClean/LandCover_HUC8_reconciliated.csv', converters={'HUC_8': lambda x: str(x), 'HUC_6': lambda x: str(x)})
LandCover_dfHUC8 = LandCover_dfHUC8.set_index('HUC_8')

AgriCensus_dfHUC6 = pd.read_csv('DatabasesClean/Agricensus_reconciliated.csv', converters={'HUC_6': lambda x: str(x)})
AgriCensus_dfHUC6 = AgriCensus_dfHUC6.set_index('HUC_6')
AgriCensus_dfHUC6 = AgriCensus_dfHUC6.fillna(0)

poly  = geopandas.GeoDataFrame.from_file('watershed/huc8sum.shp')
poly['HUC_8'] = poly['HUC_8'].astype(str)
AreaHUC8 = poly.set_index('HUC_8', drop=False)[['AREA_ACRES','HUC_8']]



# Import databases
PlantNutrientUptake_df = pd.read_csv('DatabasesClean/PlantNutrientUptake.csv', index_col='Crop')
ForestsPlantNutrientUptake = pd.read_csv('DatabasesClean/ForestsPlantNutrientUptake.csv', index_col='Item')
SmallGrainsPlantNutrientUptake = pd.read_csv('DatabasesClean/SmallGrainsPlantNutrientUptake.csv', index_col='Item')
VegetablesPlantNutrientUptake = pd.read_csv('DatabasesClean/Vegetables_GreenhousePlantNutrientUptake.csv', index_col='Item')
OtherCropsPlantNutrientUptake = pd.read_csv('DatabasesClean/OtherCropsPlantNutrientUptake.csv', index_col='Item')
PasturePlantNutrientUptake = pd.read_csv('DatabasesClean/PasturePlantNutrientUptake.csv', index_col='Item')

Wetlands_Preg = 0.77*1E-3/0.0002471052 #Kg/Acre

TotalHUC_PUptake_totalHUCs = []
AnthropogenicHUC_PUptake_totalHUCs = []
Puptake_PFOR_HUC_totalHUCs = []
Puptake_PWETL_HUC_totalHUCs = []
Puptake_PAGC_HUC_totalHUCs = []
Puptake_PAGP_HUC_totalHUCs = []
Puptake_PDEV_HUC_totalHUCs = []
Area_HUC_totalHUCs = []
counter_HUC8 = []

HUC_8_array =  AreaHUC8['HUC_8'].to_numpy()
HUC_8_array_iter = iter(HUC_8_array)
for HUC8 in HUC_8_array:
    HUC8ContPoint = HUC8
    HUC6ContPoint = HUC8ContPoint[:6]
    PUptake(HUC8ContPoint, HUC6ContPoint)
    counter_HUC8 = HUC8
    
AnthropogenicHUC_PUptake_df = pd.DataFrame(AnthropogenicHUC_PUptake_totalHUCs, columns =['AnthropogenicHUC_PUptake_year'])
AnthropogenicHUC_PUptake_df['AnthropogenicHUC_PUptake_day'] = AnthropogenicHUC_PUptake_df['AnthropogenicHUC_PUptake_year']/365
AnthropogenicHUC_PUptake_df['HUC_8'] = HUC_8_array
AnthropogenicHUC_PUptake_df['AREA_ACRES'] = Area_HUC_totalHUCs
AnthropogenicHUC_PUptake_df['PFOR_year'] = Puptake_PFOR_HUC_totalHUCs
AnthropogenicHUC_PUptake_df['PWETL_year'] = Puptake_PWETL_HUC_totalHUCs
AnthropogenicHUC_PUptake_df['PAGC_year'] = Puptake_PAGC_HUC_totalHUCs
AnthropogenicHUC_PUptake_df['PAGP_year'] = Puptake_PAGP_HUC_totalHUCs
AnthropogenicHUC_PUptake_df['PDEV_year'] = Puptake_PDEV_HUC_totalHUCs
AnthropogenicHUC_PUptake_df['PFOR_day'] = AnthropogenicHUC_PUptake_df['PFOR_year']/365
AnthropogenicHUC_PUptake_df['PWETL_day'] = AnthropogenicHUC_PUptake_df['PWETL_year']/365
AnthropogenicHUC_PUptake_df['PAGC_day'] = AnthropogenicHUC_PUptake_df['PAGC_year']/365
AnthropogenicHUC_PUptake_df['PAGP_day'] = AnthropogenicHUC_PUptake_df['PAGP_year']/365
AnthropogenicHUC_PUptake_df['PDEV_day'] = AnthropogenicHUC_PUptake_df['PDEV_year']/365
AnthropogenicHUC_PUptake_df.to_csv('DatabasesClean/AnthropogenicHUC_PUptake.csv', index=False)

In [41]:
AnthropogenicHUC_PUptake_df

,AnthropogenicHUC_PUptake_year,AnthropogenicHUC_PUptake_day,HUC_8,AREA_ACRES,PFOR_year,PWETL_year,PAGC_year,PAGP_year,PDEV_year,PFOR_day,PWETL_day,PAGC_day,PAGP_day,PDEV_day
0,6.756359e+05,1851.057271,04150306,5.512482e+05,3.644251e+06,2.685414e+05,2.856221e+05,121472.445026,0.0,9984.249315,735.729787,782.526265,332.801219,0.0
1,2.562009e+06,7019.203511,05010001,1.648037e+06,1.125917e+07,1.458824e+05,1.620258e+06,795869.238587,0.0,30847.046818,399.677915,4439.061930,2180.463667,0.0
2,1.023203e+06,2803.296401,05050002,1.078369e+06,7.153849e+06,1.642631e+03,2.096827e+05,811877.839778,0.0,19599.586344,4.500360,574.473193,2224.322849,0.0
3,1.913105e+07,52413.831190,05080001,1.598491e+06,1.255903e+06,1.394355e+04,1.863218e+07,484928.062649,0.0,3440.830710,38.201499,51047.059657,1328.570035,0.0
4,2.602242e+06,7129.430249,05090203,9.003527e+05,3.810897e+06,7.052224e+03,1.852755e+06,742434.553670,0.0,10440.813502,19.321162,5076.041818,2034.067270,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2099,4.588387e+05,1257.092276,17110014,6.449780e+05,3.426480e+06,3.929754e+04,3.546130e+05,64928.124046,0.0,9387.616486,107.664483,971.542522,177.885271,0.0
2100,1.953491e+05,535.202986,05030104,7.067925e+04,3.505338e+05,8.732932e+02,1.634456e+05,31030.211951,0.0,960.366542,2.392584,447.796122,85.014279,0.0
2101,8.210399e+06,22494.245017,10250001,1.104509e+06,1.255091e+04,1.175254e+04,8.130558e+06,68088.400395,0.0,34.386066,32.198737,22275.502718,186.543563,0.0
2102,4.944236e+06,13545.852958,10190011,8.912756e+05,1.839106e+05,3.437159e+04,4.819395e+06,90470.172548,0.0,503.864594,94.168736,13203.820736,247.863486,0.0


In [43]:
from folium.plugins import MousePosition, Search
import branca

HUC8_json = poly.to_crs(epsg='4326').to_json()

AnthropogenicHUC_PUptake_df = pd.read_csv('DatabasesClean/AnthropogenicHUC_PUptake.csv', converters={'HUC_8': lambda x: str(x)})
AnthropogenicHUC_PUptake_df_GEOjson = poly.merge(AnthropogenicHUC_PUptake_df, on='HUC_8')

m_Anthropogenic_PUptake = folium.Map(location=[48, -102], zoom_start=6)

bins = list(AnthropogenicHUC_PUptake_df_GEOjson['AnthropogenicHUC_PUptake_year'].quantile([0, 0.1, 0.2, 0.3,0.5,0.6,0.7,0.8,0.9, 1]))
# colormap = branca.colormap.linear.YlGnBu_09.scale(0, 30)

# colormap = branca.colormap.linear.YlOrRd_09.scale(0, 8500)
# colormap = colormap.to_step(index=[0, 1000, 3000, 5000, 8500])
# colormap.caption = 'Incidents of Crime in Victoria (year ending June 2018)'
# colormap.add_to(world_map)

colormap = branca.colormap.linear.YlGnBu_09.scale(
    AnthropogenicHUC_PUptake_df_GEOjson['AnthropogenicHUC_PUptake_year'].min(),
    AnthropogenicHUC_PUptake_df_GEOjson['AnthropogenicHUC_PUptake_year'].max())

# Anthropogenic_P_Uptake_layer = folium.Choropleth(
#     geo_data=HUC8_json,
#     name='Anthropogenic P Uptake',
#     data=AnthropogenicHUC_PUptake_df_GEOjson,
#     columns=['HUC_8', 'AnthropogenicHUC_PUptake_year'],
#     key_on='feature.properties.HUC_8',
# #     bins = bins,
#     fill_color=colorscale,
#     fill_opacity=0.7,
#     line_opacity=0.2,
#     nan_fill_color='grey',
#     nan_fill_opacity=0.7,
#     legend_name=r"Anthropogenic P uptake (kg/year)",
#     highlight=True,
# ).add_to(m_Anthropogenic_PUptake)

# folium.GeoJson(
#     AnthropogenicHUC_PUptake_df_GEOjson.to_json(),
#     style_function=lambda feature: {
#         'fillOpacity': 0,
#         'color': 'black',
#         'weight': 0.1,
#     },
#     tooltip=folium.features.GeoJsonTooltip(
#         fields=['HUC_8', 'AnthropogenicHUC_PUptake_year'],
#         aliases=['HUC8 watershed', 'Anthropogenic P uptake (kg/year)'], 
#         localize=True)
#     ).add_to(m_Anthropogenic_PUptake)



AnthropogenicHUC_PUptake_year_dict = AnthropogenicHUC_PUptake_df_GEOjson.set_index('HUC_8')['AnthropogenicHUC_PUptake_year']

folium.GeoJson(
    AnthropogenicHUC_PUptake_df_GEOjson.to_json(),
    name='Anthropogenic P Uptake',
    style_function=lambda feature: {
        'fillColor': colormap(AnthropogenicHUC_PUptake_year_dict[feature['properties']['HUC_8']]),
        'fillOpacity': 0.9,
        'color': 'black',
        'weight': 0.1,
    },
    tooltip=folium.features.GeoJsonTooltip(
        fields=['HUC_8', 'AnthropogenicHUC_PUptake_year'],
        aliases=['HUC8 watershed', 'Anthropogenic P uptake (kg/year)'], 
        localize=True)
    ).add_to(m_Anthropogenic_PUptake)

colormap.caption = 'Anthropogenic P uptake (kg/year)'
colormap.add_to(m_Anthropogenic_PUptake)

folium.LayerControl().add_to(m_Anthropogenic_PUptake)

formatter = "function(num) {return L.Util.formatNum(num, 3) + ' º ';};"

MousePosition(
    position='bottomright',
    separator=' | ',
    empty_string='NaN',
    lng_first=True,
    num_digits=20,
    prefix='Coordinates:',
    lat_formatter=formatter,
    lng_formatter=formatter,
).add_to(m_Anthropogenic_PUptake)

# HUC8search = Search(
#     layer=Anthropogenic_P_Uptake_layer,
#     geom_type='Polygon',
#     placeholder='Search for a HUC8 watershed',
#     collapsed=False,
#     search_label='HUC_8',
#     weight=3
# ).add_to(m_Anthropogenic_PUptake)

m_Anthropogenic_PUptake.save('m_Anthropogenic_PUptake.html')

In [ ]:
# AnthropogenicHUC_PUptake_df = pd.read_csv('DatabasesClean/AnthropogenicHUC_PUptake.csv', converters={'HUC_8': lambda x: str(x)})
# AnthropogenicHUC_PUptake_df_GEOjson = poly.merge(AnthropogenicHUC_PUptake_df, on='HUC_8')
# # AnthropogenicHUC_PUptake_df_GEOjson = AnthropogenicHUC_PUptake_df 
# # AnthropogenicHUC_PUptake_df_GEOjson['geometry'] = poly['geometry']
# # AnthropogenicHUC_PUptake_df_GEOjson
# AnthropogenicHUC_PUptake_df_GEOjson

# Antrophogenic P emissions

Agricultural emissions are main source of antrophogenic P emissions, due to commercial fertilizers application and the use of livestock organic waste nutrients from in croplands or uncontroled runoff to waterbodies. We have considered the other possible human-based sources of P emissions negigible compared with the agricultural releases (Lee and Bakshi, 2019. Energy-Water-CO2 Nexus of Fossil Fuel Based Power Generation, book carbon management, David A. Dzombak 2011. Nutrient Control in Large-Scale
U.S. Watersheds. The Chesapeake Bay and Northern Gulf of Mexico. The BRIDGE, NATIONAL ACADEMY OF ENGINEERING).

## Commercial fertilizers
### Nutrients content
Phosphorus and nitrogen content of several commercial phosphate fertilizers (Ohio State University (OSU) Extension, Ohio Agronomy Guide, 15th Edition, 2017.)

|      Fertilizer Material     | % P (as P2O5)* |  % N  |
|:----------------------------:|:--------------:|:-----:|
|     Single superphosphate    |      16-20     |   -   |
|     Triple superphosphate    |      44-48     |   -   |
| Monoammonium phosphate (MAP) |      48-61     | 10-12 |
|  Diammonium phosphate (DAP)  |       46       |   18  |
|    Ammonium polyphosphate    |       34       |   10  |

*: Phosphate is typically measured as P2O5.


The following values based on Ohio State University, 2019 are considered (Ohio State University (OSU) Extension, Ohio Agronomy Guide, 15th Edition, 2017.):

|      Fertilizer Material     | % P (as P2O5)* | % N |
|:----------------------------:|:--------------:|:---:|
|     Single superphosphate    |       18       |  -  |
|     Triple superphosphate    |       46       |  -  |
| Monoammonium phosphate (MAP) |      54.5      |  11 |
|  Diammonium phosphate (DAP)  |       46       |  18 |
|    Ammonium polyphosphate    |       34       |  10 |

*: Phosphate is typically measured as P2O5.

### Quantities
The amount of nutirents from manure for the United States were drawn from the IPNI NuGIS project database, which includes estimates based on fertilizers sales for 2014 provided by the Association of American Plant Food Control Officials (AAPFCO), fiting the data to the HUC8 watershed boundaries.

In [44]:
Fertilizer_df = pd.read_csv('PInputs/PInputs.csv', converters={'HUC_CODE': lambda x: str(x)})
for index, row in Fertilizer_df.iterrows():
    if len(row['HUC_CODE'])<8:
        row['HUC_CODE'] = '0' + row['HUC_CODE']
        Fertilizer_df.loc[index, 'HUC_CODE'] = row['HUC_CODE']
        
NutFertilizer = Fertilizer_df[['HUC_CODE', 'Tons_N_Farm_Fert', 'Tons_P2O5_Farm_Fert', 'Tons_K2O_Farm_Fert']]

# P2O5 to P
NutFertilizer['P_Fert'] = NutFertilizer['Tons_P2O5_Farm_Fert']/2.29

# K2O to K
NutFertilizer['K_Fert'] = NutFertilizer['Tons_K2O_Farm_Fert']/1.2

NutFertilizer['N_Fert'] = NutFertilizer['Tons_N_Farm_Fert']

#kg (from pound tons)
NutFertilizer[['N_Fert', 'P_Fert', 'K_Fert']] = NutFertilizer [['N_Fert', 'P_Fert', 'K_Fert']].apply(lambda x: x*2000*0.4535924)
        
NutFertilizer.to_csv('DatabasesClean/NutFertilizer.csv', index=False)
NutFertilizer

/home/emh/anaconda3/envs/EPAtool/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/emh/anaconda3/envs/EPAtool/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/home/emh/anaconda3/envs/EPAtool/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

,HUC_CODE,Tons_N_Farm_Fert,Tons_P2O5_Farm_Fert,Tons_K2O_Farm_Fert,P_Fert,K_Fert,N_Fert
0,01010001,571.016454,389.809255,595.895728,1.544232e+05,4.504896e+05,5.180174e+05
1,01010002,89.161934,60.834240,93.633475,2.409952e+04,7.078572e+04,8.088635e+04
2,01010003,679.179020,463.770211,707.130823,1.837228e+05,5.345819e+05,6.161409e+05
3,01010004,2411.329132,1646.452618,2512.172053,6.522431e+05,1.899170e+06,2.187521e+06
4,01010005,1206.046156,823.535862,1255.681324,3.262442e+05,9.492792e+05,1.094107e+06
...,...,...,...,...,...,...,...
2102,18100100,766.497153,169.463072,177.629650,6.713289e+04,1.342858e+05,6.953546e+05
2103,18100201,3362.324144,891.917249,778.587768,3.533335e+05,5.886025e+05,3.050249e+06
2104,18100202,1058.373021,194.988173,333.695054,7.724468e+04,2.522692e+05,9.601399e+05
2105,18100203,1541.330329,340.562668,427.123030,1.349141e+05,3.228996e+05,1.398271e+06


In [45]:
NutFertilizer.set_index('HUC_CODE').loc['03090203']


Tons_N_Farm_Fert          1.118705
Tons_P2O5_Farm_Fert       0.214263
Tons_K2O_Farm_Fert        0.832603
P_Fert                   84.880501
K_Fert                  629.437106
N_Fert                 1014.872269
Name: 03090203, dtype: float64

## Manure
### Nutrients content
Phosphorus and nitrogen content of several manure types (Ohio State University (OSU) Extension, Ohio Agronomy Guide, 14th Edition, 2005.)

|  Manure Type | % P (as P2O5)* | % N* |
|:------------:|:--------------:|------|
| Dairy cattle |       1.8      | 4.3  |
|    Swine**   |       4.3      | 14.0 |
|  Poultry***  |        7       | 3.9  |

*: Percentages on a dry weight basis; **: Finishing; ***: Layers

### Quantities
The amount of nutirents from manure for the United States were drawn from the IPNI NuGIS project database, which includes estimates based on livestock numbers reported in census data, and reported excretion rates for different livestock. The data reported are estimated for 2014 adpating the very detailed methodology proposed by Kellog et al. (2000) (Manure Nutrients Relative to the Capacity of Cropland and Pastureland to Assimilate Nutrients, USDA, 2000) to the HUC8 watershed boundaries.

In [46]:
ManureGenerated_df = pd.read_csv('PInputs/PInputs.csv', converters={'HUC_CODE': lambda x: str(x)})
for index, row in ManureGenerated_df.iterrows():
    if len(row['HUC_CODE'])<8:
        row['HUC_CODE'] = '0' + row['HUC_CODE']
        ManureGenerated_df.loc[index, 'HUC_CODE'] = row['HUC_CODE']
        
NutManureGenerated = ManureGenerated_df[['HUC_CODE', 'Tons_N_Exc', 'Tons_P2O5_Exc', 'Tons_K2O_Exc']]

# P2O5 to P
NutManureGenerated['P_Exc'] = NutManureGenerated['Tons_P2O5_Exc']/2.29

# K2O to K
NutManureGenerated['K_Exc'] = NutManureGenerated['Tons_K2O_Exc']/1.2

NutManureGenerated['N_Exc'] = NutManureGenerated['Tons_N_Exc']

#kg (from pound tons)
NutManureGenerated[['N_Exc', 'P_Exc', 'K_Exc']] = NutManureGenerated [['N_Exc', 'P_Exc', 'K_Exc']].apply(lambda x: x*2000*0.4535924)
        
NutManureGenerated.to_csv('DatabasesClean/NutManureGenerated.csv', index=False)
NutManureGenerated

/home/emh/anaconda3/envs/EPAtool/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/emh/anaconda3/envs/EPAtool/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/home/emh/anaconda3/envs/EPAtool/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

,HUC_CODE,Tons_N_Exc,Tons_P2O5_Exc,Tons_K2O_Exc,P_Exc,K_Exc,N_Exc
0,01010001,48.801387,28.370166,34.065434,1.123886e+04,2.575304e+04,4.427188e+04
1,01010002,9.421921,4.839030,6.576900,1.916985e+03,4.972053e+03,8.547424e+03
2,01010003,49.392404,30.229807,34.477989,1.197556e+04,2.606492e+04,4.480804e+04
3,01010004,182.389360,109.508245,127.315495,4.338175e+04,9.624890e+04,1.654609e+05
4,01010005,87.708127,53.680313,61.223986,2.126549e+04,4.628456e+04,7.956748e+04
...,...,...,...,...,...,...,...
2102,18100100,6750.896834,3165.703663,5237.419346,1.254095e+06,3.959423e+06,6.124311e+06
2103,18100201,5291.670807,2739.308467,4105.335891,1.085179e+06,3.103582e+06,4.800523e+06
2104,18100202,106.546349,97.898955,82.659819,3.878273e+04,6.248978e+04,9.665723e+04
2105,18100203,695.870124,413.954038,539.863628,1.639881e+05,4.081301e+05,6.312828e+05


In [48]:
# df["ID"] = '>' + df["ID"]
# # a = ManureGenerated_df['HUC_CODE'].where(len(ManureGenerated_df['HUC_CODE'])<8,ManureGenerated_df['HUC_CODE'])
# for index, row in ManureGenerated_df.iterrows():
#     if len(row['HUC_CODE'])<8:
#         row['HUC_CODE'] = '0' + row['HUC_CODE']
#         ManureGenerated_df.loc[index, 'HUC_CODE'] = row['HUC_CODE']
#         print(row['HUC_CODE'])

# NutManureGenerated_i = ManureGenerated_df[['HUC_CODE', 'Tons_N_Exc', 'Tons_P2O5_Exc', 'Tons_K2O_Exc']]

# # P2O5 to P
# NutManureGenerated_i['Tons_P_Exc'] = NutManureGenerated_i['Tons_P2O5_Exc']/2.29

# # K2O to K
# NutManureGenerated_i['Tons_K_Exc'] = NutManureGenerated_i['Tons_K2O_Exc']/1.2

# #kg
# NutManureGenerated = NutManureGenerated_i [['Tons_N_Exc', 'Tons_P2O5_Exc', 'Tons_K2O_Exc','Tons_P_Exc','Tons_K_Exc']].apply(lambda x: x*0.4535924)


        
# # AnthropogenicHUC_PUptake_df.to_csv('DatabasesClean/AnthropogenicHUC_PUptake.csv', index=False)
# NutManureGenerated

# # z = ('0' + ManureGenerated_df['HUC_CODE']).where(len(ManureGenerated_df['HUC_CODE'])<8 ,  ManureGenerated_df['HUC_CODE'])

## Total antrophogenic P emissions emissions

In [49]:
NutFertilizer_df = pd.read_csv('DatabasesClean/NutFertilizer.csv', converters={'HUC_CODE': lambda x: str(x)})
NutManureGenerated_df = pd.read_csv('DatabasesClean/NutManureGenerated.csv', converters={'HUC_CODE': lambda x: str(x)})

# TotalEmissions['P'] = NutManureGenerated_df['P_Exc'] + NutFertilizer_df['P_Fert']
TotalEmissions = pd.DataFrame({'HUC_8':NutManureGenerated_df['HUC_CODE'],
                               'P':(NutManureGenerated_df['P_Exc'] + NutFertilizer_df['P_Fert']),
                               'N':(NutManureGenerated_df['N_Exc'] + NutFertilizer_df['N_Fert']),
                               'K':(NutManureGenerated_df['K_Exc'] + NutFertilizer_df['K_Fert']),
                               'PManure':(NutManureGenerated_df['P_Exc']),
                               'NManure':(NutManureGenerated_df['N_Exc']),
                               'KManure':(NutManureGenerated_df['K_Exc']),
                               'PFert':(NutFertilizer_df['P_Fert']),
                               'NFert':(NutFertilizer_df['N_Fert']),
                               'KFert':(NutFertilizer_df['K_Fert']),
                                })

TotalEmissions.to_csv('DatabasesClean/TotalEmissions.csv', index=False)
AnthropogenicHUC_PUptake_df['AnthropogenicHUC_PUptake_year']-2

0       6.756339e+05
1       2.562007e+06
2       1.023201e+06
3       1.913105e+07
4       2.602240e+06
            ...     
2099    4.588367e+05
2100    1.953471e+05
2101    8.210397e+06
2102    4.944234e+06
2103    1.332584e+07
Name: AnthropogenicHUC_PUptake_year, Length: 2104, dtype: float64

# Antrophogenic P balance and TES

In [50]:
TotalEmissions_df = pd.read_csv('DatabasesClean/TotalEmissions.csv', converters={'HUC_8': lambda x: str(x)})
AnthropogenicHUC_PUptake_df = pd.read_csv('DatabasesClean/AnthropogenicHUC_PUptake.csv', converters={'HUC_8': lambda x: str(x)})

AntrophogenicPBalance = pd.DataFrame({'HUC_8':TotalEmissions_df['HUC_8'],
                                       'P_emitted':TotalEmissions_df['P'],
                                       'P_emittedManure':TotalEmissions_df['PManure'],
                                       'P_emittedFert':TotalEmissions_df['PFert'],
                                       'P_uptake':AnthropogenicHUC_PUptake_df['AnthropogenicHUC_PUptake_year'],
                                       'P_balance':(TotalEmissions_df['P'] - AnthropogenicHUC_PUptake_df['AnthropogenicHUC_PUptake_year']),
                                       'P_TES':((AnthropogenicHUC_PUptake_df['AnthropogenicHUC_PUptake_year']-TotalEmissions_df['P'])/TotalEmissions_df['P'])})

def TES(x):
    if x<=0:
        return 'Saturated'
    elif x>0:
        return 'Not saturated'
    
AntrophogenicPBalance["P Saturation"] = AntrophogenicPBalance['P_TES'].apply(TES)

AntrophogenicPBalance.to_csv('DatabasesClean/AntrophogenicPBalance.csv', index=False)
AntrophogenicPBalance

,HUC_8,P_emitted,P_emittedManure,P_emittedFert,P_uptake,P_balance,P_TES,P Saturation
0,01010001,1.656620e+05,1.123886e+04,1.544232e+05,6.756359e+05,-5.099739e+05,3.078400,Not saturated
1,01010002,2.601650e+04,1.916985e+03,2.409952e+04,2.562009e+06,-2.535993e+06,97.476312,Not saturated
2,01010003,1.956984e+05,1.197556e+04,1.837228e+05,1.023203e+06,-8.275048e+05,4.228470,Not saturated
3,01010004,6.956249e+05,4.338175e+04,6.522431e+05,1.913105e+07,-1.843542e+07,26.501961,Not saturated
4,01010005,3.475097e+05,2.126549e+04,3.262442e+05,2.602242e+06,-2.254732e+06,6.488258,Not saturated
...,...,...,...,...,...,...,...,...
2102,18100100,1.321228e+06,1.254095e+06,6.713289e+04,4.944236e+06,-3.623008e+06,2.742152,Not saturated
2103,18100201,1.438512e+06,1.085179e+06,3.533335e+05,1.332584e+07,-1.188733e+07,8.263630,Not saturated
2104,18100202,1.160274e+05,3.878273e+04,7.724468e+04,NaN,NaN,NaN,None
2105,18100203,2.989022e+05,1.639881e+05,1.349141e+05,NaN,NaN,NaN,None


In [51]:
AntrophogenicPBalance_df = pd.read_csv('DatabasesClean/AntrophogenicPBalance.csv', converters={'HUC_8': lambda x: str(x)})

AntrophogenicPBalance_stats = AntrophogenicPBalance_df
AntrophogenicPBalance_stats['P_TES_stats'] = (AntrophogenicPBalance_stats['P_TES'] > 0)
(AntrophogenicPBalance_stats['P_TES_stats']==False).sum()
# AntrophogenicPBalance_stats.describe()
697/2104*100

33.127376425855516

In [52]:
from folium.plugins import MousePosition, Search
import branca
import branca.colormap as cm

poly  = geopandas.GeoDataFrame.from_file('watershed/huc8sum.shp')
poly['HUC_8'] = poly['HUC_8'].astype(str)
HUC8_json = poly.to_crs(epsg='4326').to_json()

AntrophogenicPBalance_df = pd.read_csv('DatabasesClean/AntrophogenicPBalance.csv', converters={'HUC_8': lambda x: str(x)}).fillna(np.nan)
AntrophogenicPBalance_df_GEOjson = poly.merge(AntrophogenicPBalance_df, on='HUC_8', how = 'left').fillna(np.nan)

m_AntrophogenicPBalance = folium.Map(location=[48, -102], tiles="openstreetmap", zoom_start=6)

# bins = list(AntrophogenicPBalance_df_GEOjson['P_TES'].quantile([0, 0.1, 0.2, 0.3,0.5,0.6,0.7,0.8,0.9, 1]))
# colormap = branca.colormap.linear.YlGnBu_09.scale(0, 30)

# colormap = branca.colormap.linear.YlOrRd_09.scale(0, 8500)
# colormap = colormap.to_step(index=[0, 1000, 3000, 5000, 8500])
# colormap.caption = 'Incidents of Crime in Victoria (year ending June 2018)'
# colormap.add_to(world_map)

# colormap = branca.colormap.linear.YlGnBu_09.scale(
#     AntrophogenicPBalance_df_GEOjson['P_TES'].min(),
#     AntrophogenicPBalance_df_GEOjson['P_TES'].max())

# colormap = cm.LinearColormap(colors=['red','blue'], index=[AntrophogenicPBalance_df_GEOjson['P_TES'].min(),0],
#                              vmin=AntrophogenicPBalance_df_GEOjson['P_TES'].min(),vmax=AntrophogenicPBalance_df_GEOjson['P_TES'].max())

colormap = cm.StepColormap(colors=['#AD1E2E','#2A67A6'], index=[AntrophogenicPBalance_df_GEOjson['P_TES'].min(),0],
                             vmin=AntrophogenicPBalance_df_GEOjson['P_TES'].min(),vmax=AntrophogenicPBalance_df_GEOjson['P_TES'].max())

# colormap.to_step(index=[-200, 0, 1200])


# Anthropogenic_P_Uptake_layer = folium.Choropleth(
#     geo_data=HUC8_json,
#     name='Anthropogenic P Uptake',
#     data=AnthropogenicHUC_PUptake_df_GEOjson,
#     columns=['HUC_8', 'AnthropogenicHUC_PUptake_year'],
#     key_on='feature.properties.HUC_8',
# #     bins = bins,
#     fill_color=colorscale,
#     fill_opacity=0.7,
#     line_opacity=0.2,
#     nan_fill_color='grey',
#     nan_fill_opacity=0.7,
#     legend_name=r"Anthropogenic P uptake (kg/year)",
#     highlight=True,
# ).add_to(m_Anthropogenic_PUptake)

# folium.GeoJson(
#     AnthropogenicHUC_PUptake_df_GEOjson.to_json(),
#     style_function=lambda feature: {
#         'fillOpacity': 0,
#         'color': 'black',
#         'weight': 0.1,
#     },
#     tooltip=folium.features.GeoJsonTooltip(
#         fields=['HUC_8', 'AnthropogenicHUC_PUptake_year'],
#         aliases=['HUC8 watershed', 'Anthropogenic P uptake (kg/year)'], 
#         localize=True)
#     ).add_to(m_Anthropogenic_PUptake)



P_TES_dict = AntrophogenicPBalance_df_GEOjson.set_index('HUC_8')['P_TES']

AntrophogenicPBalance_layer = folium.GeoJson(
    AntrophogenicPBalance_df_GEOjson.to_json(),
    name='P TES',
    style_function=lambda feature: {
        'fillColor': 'grey' if np.isnan(P_TES_dict[feature['properties']['HUC_8']]) == True else colormap(P_TES_dict[feature['properties']['HUC_8']]),
        'fillOpacity': 0.9,
        'color': 'black',
        'weight': 0.9,
    },
    tooltip=folium.features.GeoJsonTooltip(
        fields=['HUC_8', 'P_TES'],
        aliases=['HUC8 watershed', 'P TES'], 
        localize=True)
    ).add_to(m_AntrophogenicPBalance)

colormap.caption = 'P TES'
colormap.add_to(m_AntrophogenicPBalance)

folium.LayerControl().add_to(m_AntrophogenicPBalance)

formatter = "function(num) {return L.Util.formatNum(num, 3) + ' º ';};"

MousePosition(
    position='bottomright',
    separator=' | ',
    empty_string='NaN',
    lng_first=True,
    num_digits=20,
    prefix='Coordinates:',
    lat_formatter=formatter,
    lng_formatter=formatter,
).add_to(m_AntrophogenicPBalance)

HUC8search = Search(
    layer=AntrophogenicPBalance_layer,
    geom_type='Polygon',
    placeholder='Search for a HUC8 watershed',
    collapsed=False,
    search_label='HUC_8',
    weight=3
).add_to(m_AntrophogenicPBalance)

m_AntrophogenicPBalance.save('m_AntrophogenicPBalance.html')
# m_AntrophogenicPBalance._to_png(delay=15)